In [1]:
import numpy as np 
import pickle
import casadi as cs
import matplotlib.pyplot as plt 

# MPC

## Load simulated data

In [2]:
def load():
    f = open(
        '/home/lars/ros/crane_ws/src/crane/crane_controllers/data/datain.pickle', 'rb')
    datain = pickle.load(f)
    f.close()
    f = open(
        '/home/lars/ros/crane_ws/src/crane/crane_controllers/data/dataout.pickle', 'rb')
    dataout = pickle.load(f)
    f.close()
    return datain, dataout

In [3]:
datain, dataout = load()

last_gopts = datain['last_gopt']
last_gs = datain['last_g']
Ls = datain['L']
ks = datain['k']
gammas = datain['gamma']
ts = datain['t']
zrefs = datain['zref']
zs = datain['z']

dot_wxs = dataout['dot_wx']
dot_wys = dataout['dot_wy']
gxs = dataout['gx']
gys = dataout['gy']

In [28]:
def cs_continuous_dynamics():
    kp = cs.SX.sym('kp')
    kd = cs.SX.sym('kd')
    k = cs.vertcat(kp, kd)
    
    L = cs.SX.sym('L')

    x0 = cs.SX.sym('x0')
    dx0 = cs.SX.sym('dx0')
    y0 = cs.SX.sym('y0')
    dy0 = cs.SX.sym('dy0')
    phix = cs.SX.sym('phix')
    dphix = cs.SX.sym('dphix')
    phiy = cs.SX.sym('phiy')
    dphiy = cs.SX.sym('dphiy')
    z = cs.vertcat(x0, dx0, y0, dy0, phix, dphix, phiy, dphiy)
    
    cx = cs.SX.cos(phix)
    sx = cs.SX.sin(phix)
    cy = cs.SX.cos(phiy)
    sy = cs.SX.sin(phiy)
    
    gx = cs.SX.sym('gx')
    gy = cs.SX.sym('gy')
    g = cs.vertcat(gx, gy)

    dzdt = cs.SX.sym('dzdt', 8,1)

    uy = - L*cy/cx*(kd*dphix + kp*phix) - 2.0*L*sy / \
        cx*dphix*dphiy + 9.81*sx*sy*sy/cx

    # Eq 51-54 in Ecc
    dzdt[0] = dx0  # dx0
    dzdt[1] = gx + (L*(kd*dphiy + kp*phiy) - L*cy*sy *
                    dphix*dphix - uy*sx*sy)/cy  # ddx0
    dzdt[2] = dy0  # dy0
    dzdt[3] = gy - (L*cy*(kd*dphix + kp*phix) + 2.0*L*sy *
                    dphix*dphiy - 9.81*sx*sy*sy)/cx  # ddy0
    dzdt[4] = z[5]  # dphix
    dzdt[5] = (-kd*dphix) - (kp*phix) - \
        (9.81/L*cy*sx) + (gy*cx/(cy*L))  # ddphix
    dzdt[6] = z[7]  # dphiy
    dzdt[7] = (-kd*dphiy) - (kp*phiy) - (9.81 / L * cx*sy) - \
        ((gx*cy + gy*sx*sy)/L)  # ddphiy
    
    f = cs.Function('cs', [z, g, k, L], [dzdt], ['z', 'g', 'k', 'L'], ['dzdt'])
    return f

In [32]:
def cs_discrete_dynamics():
    zk = cs.SX.sym('zk', 8)
    zk1 = cs.SX.sym('zk1', 8)
    gk = cs.SX.sym('gk', 2)
    Ts = cs.SX.sym('Ts')
    k = cs.SX.sym('k', 2)
    L = cs.SX.sym('L')
    
    cd = cs_continuous_dynamics()
    
    M = 10
    delta = Ts/M
    zk1 = zk
    for i in range(M+1):
        zk1 += delta * cd(zk1, gk, k, L)
    
    f = cs.Function('dd', [zk, gk, Ts, k, L], [zk1], ['zk', 'gk', 'Ts', 'k', 'L'], ['zk1'])
    return f

In [33]:
cd = cs_continuous_dynamics()
dd = cs_discrete_dynamics()

In [34]:
idx = 99
z = zs[idx]
g = last_gs[idx]
k = ks[idx]
L = Ls[idx]
Ts = 0.2
cd(z, g, k, L)

DM([-0.243796, 0.129262, -0.273322, 0.0403891, 0.002603, -0.463153, 0.0455681, 0.46586])

In [35]:
dd(z, g, Ts, k, L)

DM([1.15971, -0.196165, -0.180994, -0.245487, 0.0453123, -0.0764139, -0.0447872, 0.115519])